<b>Note: This Jupyter Notebook is associated with the article [The Calendar Spread Options Strategy (and How to Build with Alpaca)](https://alpaca.markets/learn/calendar-spread).</b>

# Step 1: Setting Up the Environment and Trade Parameters

In [ ]:
!python3 -m pip install --upgrade alpaca-py
import pandas as pd
import numpy as np
from scipy.stats import norm
import alpaca
import time
from scipy.optimize import brentq
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit

from alpaca.data.historical.option import OptionHistoricalDataClient
from alpaca.data.historical.stock import StockHistoricalDataClient, StockLatestTradeRequest
from alpaca.data.requests import StockBarsRequest, OptionLatestQuoteRequest, OptionChainRequest, OptionBarsRequest
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import (
    MarketOrderRequest,
    GetOptionContractsRequest,
    MarketOrderRequest,
    OptionLegRequest,
    ClosePositionRequest,
)
from alpaca.trading.enums import (
    AssetStatus,
    ExerciseStyle,
    OrderSide,
    OrderClass,
    OrderStatus,
    OrderType,
    TimeInForce,
    QueryOrderStatus,
    ContractType
)

In [ ]:
# API_KEY = "Alpaca's Trading API Key (Paper Account)"
# API_SECRET = "Alpaca's Trading API Secret Key (Paper Account)"

# A safe approach to setting up API credentials for Alpaca (Assume you run this notebook in Google Colab)
# Add your key to Colab Secrets. Add your API key to the Colab Secrets manager to securely store it
from google.colab import userdata
API_KEY = userdata.get('ALPACA_API_KEY')
API_SECRET = userdata.get('ALPACA_SECRET_KEY')
BASE_URL = None
## We use paper environment for this example (Please do not modify this. This example is for paper trading only).
PAPER = True

# Initialize Alpaca clients
trade_client = TradingClient(api_key=API_KEY, secret_key=API_SECRET, paper=PAPER, url_override=BASE_URL)
option_historical_data_client = OptionHistoricalDataClient(api_key=API_KEY, secret_key=API_SECRET, url_override=BASE_URL)
stock_data_client = StockHistoricalDataClient(api_key=API_KEY, secret_key=API_SECRET)

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
option_stream_data_wss = None

In [ ]:
# Select the underlying stock (Alcoa Corporation)
underlying_symbol = 'AA'

# Set the timezone
timezone = ZoneInfo("America/New_York")

# Get current date in US/Eastern timezone
today = datetime.now(timezone).date()

# Define a 5% range around the underlying price 
STRIKE_RANGE = 0.05

# Buying power percentage to use for the trade
BUY_POWER_LIMIT = 0.05

# Risk free rate for the options greeks and IV calculations
risk_free_rate = 0.01

# Check account buying power
buying_power = float(trade_client.get_account().buying_power)

# Set the open interest volume threshold
OI_THRESHOLD = 100

# Calculate the limit amount of buying power to use for the trade
buying_power_limit = buying_power * BUY_POWER_LIMIT

# Set the expiration date range for the options
min_expiration = today + timedelta(days=21)
max_expiration = today + timedelta(days=80)

# Get the latest price of the underlying stock
def get_underlying_price(symbol):
    # Get the latest trade for the underlying stock
    underlying_trade_request = StockLatestTradeRequest(symbol_or_symbols=symbol)
    underlying_trade_response = stock_data_client.get_stock_latest_trade(underlying_trade_request)
    return underlying_trade_response[symbol].price

# Get the latest price of the underlying stock
underlying_price = get_underlying_price(underlying_symbol)

# Set the minimum and maximum strike prices based on the underlying price
min_strike = str(underlying_price * (1 - STRIKE_RANGE))
max_strike = str(underlying_price * (1 + STRIKE_RANGE))

# Define expiry thresholds. For a shorter-term call, between 27 days and 40 days, which is buffer for 30 days; for a longer-term, between 55 days and 65 days, which is buffer for 60 days
EXPIRY_RANGE = (27, 40, 55, 65)

# Define Implied Volatility thresholds. For a shorter-term call, between 0.40 and 0.70; for a longer-term, between 0.30 and 0.60
IV_RANGE = (0.40, 0.70, 0.30, 0.60)

# Define delta thresholds. For a shorter-term call, between 0.45 and 0.80; for a longer-term, between 0.40 and 0.70
DELTA_RANGE = (0.45, 0.80, 0.40, 0.70)

# Define theta thresholds. For a shorter-term call, between -0.2 and -0.01; for a longer-term, between -0.1 and -0.01
THETA_RANGE = (-0.2, -0.01, -0.1, -0.01)

# Set target profit levels
TARGET_PROFIT_PERCENTAGE = 0.6

# Display the values
print(f"Underlying Symbol: {underlying_symbol}")
print(f"{underlying_symbol} price: {underlying_price}")
print(f"Strike Range: {STRIKE_RANGE}")
print(f"Buying Power Limit Percentage: {BUY_POWER_LIMIT}")
print(f"Risk Free Rate: {risk_free_rate}")
print(f"Account Buying Power: {buying_power}")
print(f"Buying Power Limit: {buying_power_limit}")
print(f"Open Interest Threshold: {OI_THRESHOLD}")
print(f"Minimum Expiration Date: {min_expiration}")
print(f"Maximum Expiration Date: {max_expiration}")
print(f"Minimum Strike Price: {min_strike}")
print(f"Maximum Strike Price: {max_strike}")
print(f"Expiry date range for options: {EXPIRY_RANGE}")
print(f"Implied Volatility range for options: {IV_RANGE}")
print(f"Delta range for options: {DELTA_RANGE}")
print(f"Theta range for options: {THETA_RANGE}")

# Step 2: Calculations for Implied Volatility and Option Greeks with Black-Scholes model

In [ ]:
# Calculate implied volatility
def calculate_implied_volatility(option_price, S, K, T, r, option_type):
    
    # Define a reasonable range for sigma
    sigma_lower = 1e-6
    sigma_upper = 5.0  # Adjust upper limit if necessary

    # Check if the option is out-of-the-money and price is close to zero
    intrinsic_value = max(0, (S - K) if option_type == 'call' else (K - S))
    if option_price <= intrinsic_value + 1e-6:
        
        print("Option price is close to intrinsic value; implied volatility is near zero.")
        return 0.0
    
    # Define the function to find the root
    def option_price_diff(sigma):
        d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        if option_type == 'call':
            price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        elif option_type == 'put':
            price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        return price - option_price

    try:
        return brentq(option_price_diff, sigma_lower, sigma_upper)
    except ValueError as e:
        print(f"Failed to find implied volatility: {e}")
        return None


def calculate_greeks(option_price, strike_price, expiry, underlying_price, risk_free_rate, option_type):
    T = (expiry - pd.Timestamp.now()).days / 365 # It is unconventional, but some use 225 days (# of annual trading days) in replace of 365 days
    T = max(T, 1e-6)  # Set minimum T to avoid zero
    
    if T == 1e-6:
        print("Option has expired or is expiring now; setting Greeks based on intrinsic value.")
        if option_type == 'put':
            delta = -1.0 if underlying_price < strike_price else 0.0
        else:
            delta = 1.0 if underlying_price > strike_price else 0.0
        gamma = 0.0
        theta = 0.0
        vega = 0.0
        return delta, gamma, theta, vega
    
    # Calculate IV
    IV = calculate_implied_volatility(option_price, underlying_price, strike_price, T, risk_free_rate, option_type)

    if IV is None or IV == 0.0:
        print("Implied volatility could not be determined, skipping Greek calculations.")
        return None
    
    d1 = (np.log(underlying_price / strike_price) + (risk_free_rate + 0.5 * IV ** 2) * T) / (IV * np.sqrt(T))
    d2 = d1 - IV * np.sqrt(T) # d2 for Theta calculation
    # Calculate Delta
    delta = norm.cdf(d1) if option_type == 'call' else -norm.cdf(-d1)
    # Calculate Gamma
    gamma = norm.pdf(d1) / (underlying_price * IV * np.sqrt(T))
    # Calculate Vega
    vega = underlying_price * np.sqrt(T) * norm.pdf(d1)
    # Calculate Theta
    if option_type == 'call':
        theta = (
            - (underlying_price * norm.pdf(d1) * IV) / (2 * np.sqrt(T))
            - (risk_free_rate * strike_price * np.exp(-risk_free_rate * T) * norm.cdf(d2))
        )
    else:
        theta = (
            - (underlying_price * norm.pdf(d1) * IV) / (2 * np.sqrt(T))
            + (risk_free_rate * strike_price * np.exp(-risk_free_rate * T) * norm.cdf(-d2))
        )
    # Convert annualized theta to daily theta
    theta /= 365
    
    return delta, gamma, theta, vega

# Step 3: Historical Market Data Analysis with Its Stock Chart
Factors To Include In The Strategy

In [ ]:
# Get the historical data for the underlying stock by symbol and timeframe
# ref. https://alpaca.markets/sdks/python/api_reference/data/option/historical.html
def get_stock_data(underlying_symbol, days=90):
    today = datetime.now(timezone).date()
    req = StockBarsRequest(
        symbol_or_symbols=[underlying_symbol],
        timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Day),     # specify timeframe
        start=today - timedelta(days=days),                          # specify start datetime, default=the beginning of the current day.
    )
    return stock_data_client.get_stock_bars(req).df

In [ ]:
priceData = get_stock_data(underlying_symbol, days=180)

# List of stock agg objects while dropping the symbol column
priceData = priceData.reset_index(level='symbol', drop=True)

import plotly.graph_objects as go

# Bar chart for the stock price
fig = go.Figure(data=[go.Candlestick(x=priceData.index,
                open=priceData['open'],
                high=priceData['high'],
                low=priceData['low'],
                close=priceData['close'])])

fig.show()


# Computing RVI and Bolinger Band

## Relative Volatility Index (RVI)

If RVI > 70, it is overbought. In options trading, this could suggest that implied volatility (IV) is elevated, and option premiums may be overpriced. Traders might consider selling options or strategies that benefit from a drop in volatility (e.g., selling straddles or strangles).\
If RVI < 50, it suggests that volatility is currently slightly lower than its recent average, but it could signal volatility expansion at the same time\
If RVI < 30, In options trading, this could suggest that implied volatility (IV) is depressed, and option premiums may be underpriced. Traders might consider buying options or strategies that benefit from a rise in volatility.

In [ ]:
def calculate_rvi(df, period=14):
    # Calculate daily price changes 
    df["price_change"] = df["close"].diff()

    # Separate up and down price changes
    df['up_change'] = df['price_change'].where(df['price_change'] > 0, 0)
    df['down_change'] = -df['price_change'].where(df['price_change'] < 0, 0)

    # Calculate std of up and down changes over the rolling period
    df['avg_up_std'] = df['up_change'].rolling(window=period).std()
    df['avg_down_std'] = df['down_change'].rolling(window=period).std()

    # Calculate RVI
    df['rvi'] = 100 - (100 / (1 + (df['avg_up_std'] / df['avg_down_std'])))

    # Smooth the RVI with a 4 periods moving average
    df['rvi_smoothed'] = df['rvi'].rolling(window=4).mean()

    return df['rvi_smoothed']

# smoothed RVI
calculate_rvi(priceData, period=21).iloc[30:]

## The upper Bollinger Bands of underlying stock price

In [ ]:
# setup bollinger band calculations
def check_bb(df, period=14, multiplier=2):
    bollinger_bands = []
    # Calculate the Simple Moving Average (SMA)
    df['SMA'] = df['close'].rolling(window=period).mean()
    # Calculate the rolling standard deviation
    df['StdDev'] = df['close'].rolling(window=period).std()
    # Calculate the Upper Bollinger Band (two standard deviation)
    df['Upper Band'] = df['SMA'] + (multiplier * df['StdDev'])
    # Calculate the Lower Bollinger Band (two standard deviation)
    df['Lower Band'] = df['SMA'] - (multiplier * df['StdDev'])
    # Get the most recent Upper Band value
    upper_bollinger_band = df['Upper Band'].iloc[-1]
    lower_bollinger_band = df['Lower Band'].iloc[-1]
    
    bollinger_bands = [upper_bollinger_band, lower_bollinger_band]
    return bollinger_bands

bollinger_bands = check_bb(priceData, 14, 2)

# The current market price is not too close to the two-standard deviation level yet but is relatively closer to the higher Bollinger Band.
print(f"Latest Upper Bollinger Band is: {bollinger_bands[0]}. Latest Lower Bollinger Band is {bollinger_bands[1]}; while underlying stock '{underlying_symbol}' price is {underlying_price}.")


# Step 4: Setting Up for A Long Calendar Spread with Calls

In [ ]:
# Check for call options
def get_call_options(underlying_symbol, min_strike, max_strike, min_expiration, max_expiration):
    
    # Fetch the options data to add to the portfolio
    req = GetOptionContractsRequest(
        underlying_symbols=[underlying_symbol],
        status=AssetStatus.ACTIVE,
        type=ContractType.CALL,
        strike_price_gte=min_strike,
        strike_price_lte=max_strike,
        expiration_date_gte=min_expiration,
        expiration_date_lte=max_expiration,
    )
    
    # Get call option chain of the underlying symbol
    call_options = trade_client.get_option_contracts(req).option_contracts
    return call_options

In [ ]:
def build_option_dict(option_data, iv, delta, theta, option_price):
    """
    Helper to build an option dictionary from option_data and calculated metrics.
    """
    return {
        'id': option_data.id,
        'name': option_data.name,
        'symbol': option_data.symbol,
        'strike_price': option_data.strike_price,
        'root_symbol': option_data.root_symbol,
        'underlying_symbol': option_data.underlying_symbol,
        'underlying_asset_id': option_data.underlying_asset_id,
        'close_price': option_data.close_price,
        'close_price_date': option_data.close_price_date,
        'expiration_date': option_data.expiration_date,
        'open_interest': option_data.open_interest,
        'open_interest_date': option_data.open_interest_date,
        'size': option_data.size,
        'status': option_data.status,
        'style': option_data.style,
        'tradable': option_data.tradable,
        'type': option_data.type,
        'initial_IV': iv,
        'initial_delta': delta,
        'initial_theta': theta,
        'initial_option_price': option_price,
    }

In [ ]:
def find_call_options_for_calendar_spread(call_options, underlying_price, risk_free_rate, buying_power_limit, expiry_range, iv_range, delta_range, theta_range):
    
    short_call = None
    long_call = None

    for option_data in call_options:
        option_symbol = option_data.symbol
        print(f"\nChecking option {option_symbol}...")

        try:
            # Check required fields and open interest
            if option_data.open_interest is None or option_data.open_interest_date is None:
                print(f"Skipping {option_symbol}: missing open_interest or open_interest_date.")
                continue
            if float(option_data.open_interest) <= OI_THRESHOLD:
                print(f"Skipping {option_symbol}: open_interest {option_data.open_interest} is below threshold {OI_THRESHOLD}.")
                continue

            # Get option quote and calculate basic metrics
            option_quote_request = OptionLatestQuoteRequest(symbol_or_symbols=option_symbol)
            option_quote = option_historical_data_client.get_option_latest_quote(option_quote_request)[option_symbol]

            option_price = (option_quote.bid_price + option_quote.ask_price) / 2
            strike_price = float(option_data.strike_price)
            expiry = pd.Timestamp(option_data.expiration_date)
            option_type = option_data.type.value

            print(f"Option {option_symbol}: price = {option_price}, strike = {strike_price}, expiry = {expiry}")
            remaining_days = (expiry - pd.Timestamp.now()).days
            T = max(remaining_days / 365, 1e-6)  # Avoid division by zero
            # Calculate IV of options
            iv = calculate_implied_volatility(
                option_price=option_price, 
                S=underlying_price,
                K=strike_price,
                T=T,
                r=risk_free_rate,
                option_type=option_type
            )
            # Calculate option Greeks
            delta, _, theta, _ = calculate_greeks(
                option_price=option_price, 
                strike_price=strike_price, 
                expiry=expiry,
                underlying_price=underlying_price,
                risk_free_rate=risk_free_rate,
                option_type=option_type
            )
            # Print the metrics used for filtering options
            print(f"Metrics for {option_symbol}: remaining_days = {remaining_days}, IV = {iv}, delta = {delta}, theta = {theta}")

            # Check short_call (shorter-term) criteria
            if expiry_range[0] <= remaining_days <= expiry_range[1]:
                short_conditions = [
                    (iv_range[0] <= iv <= iv_range[1], f"failed IV check for short call: {iv} not in [{iv_range[0]}, {iv_range[1]}]"),
                    (delta_range[0] <= delta <= delta_range[1], f"failed delta check for short call: {delta} not in [{delta_range[0]}, {delta_range[1]}]"),
                    (theta_range[0] <= theta <= theta_range[1], f"failed theta check for short call: {theta} not in [{theta_range[0]}, {theta_range[1]}]")
                ]
                for condition, message in short_conditions:
                    if not condition:
                        print(f"Option {option_symbol} {message}.")
                        break
                else:
                    print(f"Option {option_symbol} qualifies for short_call criteria.")
                    short_call = build_option_dict(option_data, iv, delta, theta, option_price)
                    print(f"short_call set to {option_symbol}.")
            else:
                print(f"Option {option_symbol} not in short_call expiry range [{expiry_range[0]}, {expiry_range[1]}] (remaining_days: {remaining_days}).")

            # Check long_call (longer-term) criteria
            if expiry_range[2] <= remaining_days <= expiry_range[3]:
                # If we've already found a short_call, ensure the strike prices match.
                if short_call is not None and strike_price != float(short_call.get('strike_price')):
                    print(f"Skipping {option_symbol} for long_call because strike price {strike_price} does not match short_call strike price {short_call.get('strike_price')}.")
                    continue
                long_conditions = [
                    (iv_range[2] <= iv <= iv_range[3], f"failed IV check for long_call: {iv} not in [{iv_range[2]}, {iv_range[3]}]"),
                    (delta_range[2] <= delta <= delta_range[3], f"failed delta check for long_call: {delta} not in [{delta_range[2]}, {delta_range[3]}]"),
                    (theta_range[2] <= theta <= theta_range[3], f"failed theta check for long call: {theta} not in [{theta_range[2]}, {theta_range[3]}]"),
                    (strike_price * float(option_data.size) < buying_power_limit, 
                     f"failed buying power check for long_call: {strike_price} * {option_data.size} = {strike_price * float(option_data.size)} is not less than {buying_power_limit}")
                ]
                for condition, message in long_conditions:
                    if not condition:
                        print(f"Option {option_symbol} {message}.")
                        break
                else:
                    if short_call is not None and option_data.symbol == short_call.get('symbol'):
                        print(f"Skipping {option_symbol} for long_call because it's the same as the short_call.")
                    else:
                        print(f"Option {option_symbol} qualifies for long_call criteria.")
                        long_call = build_option_dict(option_data, iv, delta, theta, option_price)
                        print(f"long_call set to {option_symbol}.")
            else:
                print(f"Option {option_symbol} not in long_call expiry range [{expiry_range[2]}, {expiry_range[3]}] (remaining_days: {remaining_days}).")

            if short_call and long_call:
                print("Both short_call and long_call selected. Breaking loop.")
                break

        except Exception as e:
            print(f"Error processing {option_symbol}: {e}")
            continue
    
    mleg_option_data = [option for option in [short_call, long_call] if option is not None]
    print(f"\nReturning combined list of options: {mleg_option_data}")

    if not mleg_option_data:
        raise Exception("No valid call options found. Halting further process.")

    return mleg_option_data

# Step 5: Execute a Long Call Calendar Spread

In [ ]:
def execute_long_call_calendar_spread(underlying_symbol, risk_free_rate, buying_power_limit, min_strike, max_strike, min_expiration, max_expiration, expiry_range, iv_range, delta_range, theta_range):
    # Get call options
    call_options = get_call_options(underlying_symbol, min_strike, max_strike, min_expiration, max_expiration)

    if call_options:
        # Get the latest price of the underlying stock
        underlying_price = get_underlying_price(symbol=underlying_symbol)
        
        # Find appropriate short and long call options
        mleg_option_data = find_call_options_for_calendar_spread(call_options, underlying_price, risk_free_rate, buying_power_limit, expiry_range, iv_range, delta_range, theta_range)

        # Proceed if short call and long call options are found
        if mleg_option_data:

            # Place orders for the spread
            # Create a list for the order request
            order_legs = []
            ## Append short call for a shorter-term
            order_legs.append(OptionLegRequest(
                symbol=mleg_option_data[0]["symbol"],
                side=OrderSide.SELL,
                ratio_qty=1
            ))
            ## Append long call for a longer-term
            order_legs.append(OptionLegRequest(
                symbol=mleg_option_data[1]["symbol"],
                side=OrderSide.BUY,
                ratio_qty=1
            ))
            # Place the order for both legs simultaneously
            req = MarketOrderRequest(
                qty=1,
                order_class=OrderClass.MLEG,
                time_in_force=TimeInForce.DAY,
                legs=order_legs
            )
            res = trade_client.submit_order(req)
            print("Long Call Calendar Spread order placed successfully.")

            success_message = (f"Placing Long Call Calendar Spread on {underlying_symbol} successfully:\n"
                               f"Sell {mleg_option_data[0]['symbol']} at (Strike: {mleg_option_data[0]['strike_price']}, Premium to Receive: {mleg_option_data[0]['initial_option_price']})\n"
                               f"Buy {mleg_option_data[1]['symbol']} at (Strike: {mleg_option_data[1]['strike_price']}, Premium to Pay: {mleg_option_data[1]['initial_option_price']})"
                               )
            return success_message, res, mleg_option_data

        else:
            return "Could not find suitable options for a calendar spread.", None, None
    else:
        return "No call options found for the underlying symbol.", None, None

In [ ]:
call_options = get_call_options(underlying_symbol, min_strike, max_strike, min_expiration, max_expiration)
mleg_option_data = find_call_options_for_calendar_spread(call_options, underlying_price, risk_free_rate, buying_power_limit, EXPIRY_RANGE, IV_RANGE, DELTA_RANGE, THETA_RANGE)

In [ ]:
# Run the `execute_long_call_calendar_spread` function to execute the long call calendar spread
message, res, call_calendar_option_data = execute_long_call_calendar_spread(underlying_symbol, risk_free_rate, buying_power_limit, min_strike, max_strike, min_expiration, max_expiration, EXPIRY_RANGE, IV_RANGE, DELTA_RANGE, THETA_RANGE)
message, res, call_calendar_option_data

# Cancelling Multi-leg Orders

You can cancel the order only when the order has not be filled.

In [ ]:
# Query by the order's id
q1 = trade_client.get_order_by_client_id(res.client_order_id)

# Replace overall order
if q1.status != OrderStatus.FILLED:
    # Cancel the whole order
    trade_client.cancel_order_by_id(res.id)
    print(f"Canceled order: {res}")

else:
    print("Order is already filled.")

# Step 6: How to Adjust or Exit a Calendar Spread (Rolling and Rinsing)

`mleg_roll_rinse_execution` is for execution of rinsing or rolling a multi-leg option strategy. By setting rolling=True, the function roll the strategy by buying the short call or selling the long call and executing the long call calendar spread on another option of the same underlying with a different strike and/or expiration date.

Since we set Asynchronous Calls in jupyter notebook, you'll need to await their completion (or use something like asyncio.gather) to ensure all positions are liquidated before re-entering the market.

In [ ]:
# Exit the market
def mleg_roll_rinse_execution(mleg_option_data, rolling, underlying_symbol, risk_free_rate, buying_power_limit, min_strike, max_strike, min_expiration, max_expiration, expiry_range, iv_range, delta_range, theta_range):

    # loop through the list and extract the "symbol" for each leg
    option_symbols = [leg.get('symbol') for leg in mleg_option_data]

    # if rolling the option, close the put and re-enter the market with a new put or close the call and re-enter the market with a new call
    if rolling:

        for option_symbol in option_symbols:
            try:
                # Close every leg by liquidating it (buying or selling it)
                trade_client.close_position(
                    symbol_or_asset_id=option_symbol,
                    close_options=ClosePositionRequest(qty="1")
                )
                print(f"Liquidated (Closed) {option_symbol} option.")
            except Exception as e:
                # Immediately halt further processing if any liquidation fails
                raise Exception(f"Liquidation failed for {option_symbol}. Error: {e}. Process halted.")
        
        # Roll the option
        message, res = execute_long_call_calendar_spread(
                underlying_symbol=underlying_symbol,
                risk_free_rate=risk_free_rate, 
                buying_power_limit=buying_power_limit, 
                min_strike=min_strike, 
                max_strike=max_strike, 
                min_expiration=min_expiration, 
                max_expiration=max_expiration, 
                expiry_range=expiry_range, 
                iv_range=iv_range, 
                delta_range=delta_range,
                theta_range=theta_range
        )
        print("Re-entered the market:") 
        return message, res

    # if we only want to close the position without rolling
    else:
        messages = []
        for option_symbol in option_symbols:
            try:
                trade_client.close_position(
                    symbol_or_asset_id=option_symbol,
                    close_options=ClosePositionRequest(qty='1')
                )
                messages.append(f"Liquidated (Closed) {option_symbol} option.")
            except Exception as e:
                messages.append(f"Failed to liquidate {option_symbol}. Error: {e}")
        return "\n".join(messages), None


In [ ]:
# mleg_option_data

short_term_option = min(mleg_option_data, key=lambda opt: opt['expiration_date'])
short_term_option['expiration_date']

In [ ]:
# check the current status of the sold option (rolling or rinsing)
def roll_rinse_option(mleg_option_data, rolling, target_profit_percentage, bollinger_bands, underlying_symbol, risk_free_rate, buying_power_limit, min_strike, max_strike, min_expiration, max_expiration, expiry_range, iv_range, delta_range, theta_range):

    # Determine the short-term option (earliest expiration date)
    short_term_option = min(mleg_option_data, key=lambda opt: opt['expiration_date'])

    for option in mleg_option_data:
        option_symbol = option.get("symbol")
        option_quote_request = OptionLatestQuoteRequest(symbol_or_symbols=option_symbol)
        option_quote = option_historical_data_client.get_option_latest_quote(option_quote_request)[option_symbol]
        
        # Extract option details
        current_option_price = (option_quote.bid_price + option_quote.ask_price) / 2
        strike_price = float(option["strike_price"])
        expiry = pd.Timestamp(option["expiration_date"])
        option_type = option["type"]
        remaining_days = (expiry - pd.Timestamp.now()).days
        T = max(remaining_days / 365, 1e-6)

        # Calculate current IV and Greeks
        current_iv = calculate_implied_volatility(
            option_price=current_option_price, 
            S=underlying_price,
            K=strike_price,
            T=T,
            r=risk_free_rate,
            option_type=option_type
        )
        current_delta, _, current_theta, _ = calculate_greeks(
            option_price=current_option_price, 
            strike_price=strike_price, 
            expiry=expiry,
            underlying_price=underlying_price,
            risk_free_rate=risk_free_rate,
            option_type=option_type
        )
        # Calcualte the target profit price based on the predefine target profit percentage (Default: 60% of credit received, meaning if you earn 60% of the initial premium, you exit)
        target_profit_price = option['initial_option_price'] - option['initial_option_price'] * target_profit_percentage

        # If the target option is the same as the shorter term option
        if short_term_option['expiration_date'] == option['expiration_date']:
            # Short-term thresholds:
            #  - Expires in less than 7 days; |delta| >= 0.7; IV >= 60; Option price is at or below the target profit price; Option price is outside the Bollinger Band range (below lower or above upper)
            exit_condition = (remaining_days < 7 or abs(current_delta) >= 0.7 or current_iv >= 60 or current_option_price <= target_profit_price or current_option_price <= bollinger_bands[1] or current_option_price >= bollinger_bands[0])

        else:
            # Longer-term call thresholds:
            #  - Expires in less than 30 days; |delta| >= 0.8; IV >= 60; Option price is at or below the target profit price; Option price is outside the Bollinger Band range (below lower or above upper)
            exit_condition = (remaining_days < 30 or abs(current_delta) >= 0.8 or current_iv >= 60 or current_option_price <= target_profit_price or current_option_price <= bollinger_bands[1] or current_option_price >= bollinger_bands[0])
                
        # Execute roll/rinse if any threshold is met.
        if exit_condition:
            message, response = mleg_roll_rinse_execution(
                mleg_option_data, rolling, underlying_symbol, risk_free_rate,
                buying_power_limit, min_strike, max_strike, min_expiration,
                max_expiration, expiry_range, iv_range, delta_range, theta_range
            )
            return message, response

    return "No option met the exit thresholds. Holding the position.", None

In [ ]:
roll_rinse_option(mleg_option_data, True, TARGET_PROFIT_PERCENTAGE, bollinger_bands, underlying_symbol, risk_free_rate, buying_power_limit, min_strike, max_strike, min_expiration, max_expiration, EXPIRY_RANGE, IV_RANGE, DELTA_RANGE, THETA_RANGE)

# Back Testing

In [ ]:
# Initialize the Option Historical Data Client
option_historical_data_client = OptionHistoricalDataClient(
    api_key=API_KEY, 
    secret_key=API_SECRET, 
    url_override=BASE_URL
)

# Define the request parameters
req = OptionBarsRequest(
    symbol_or_symbols=mleg_option_data[0]["symbol"],
    timeframe=TimeFrame.Day,  # Choose timeframe (Minute, Hour, Day, etc.)
    start="2024-09-10",  # Start date
    end="2025-02-17"  # End date
)

option_historical_data_client.get_option_bars(req)